## selenium 爬虫 - 木材网


In [1]:
import os
driver_path=''
match(os.name):
    case "nt":
        # Windows
        driver_path = r"./driver/chromedriver-win64/chromedriver.exe"  
        print("Windows")
    case "posix":
        # Mac
        driver_path = r"./driver/chromedriver-mac-arm64/chromedriver"  
        print("Mac")

os.environ["PATH"] += ";" + driver_path

Mac


### 启动 selenium


#### 启动浏览器 和 打开网站


In [2]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
driver = webdriver.Chrome(service=Service(executable_path=driver_path))
driver.get("https://www.chinatimber.org/baojia/baojia.html?prov=&bigCate=&q=")
time.sleep(3)

### 获取数据

In [4]:
import random
from selenium.webdriver.common.by import By
# 定位到倒数第二个 <li> 元素 即为最大页数
pagination_items = driver.find_elements(By.CSS_SELECTOR, "ul.pagination li")
max_page_item = pagination_items[-2] 
# 提取文本
max_page = max_page_item.text.strip()
print(f"数据最大页数为： {max_page}")

数据最大页数为： 605


In [9]:
max_page=int(5)
reduce_data = []
def get_data():
    for i in range(1, int(max_page)):
        print(f"正在抓取第{i}页")
        # 定位表格中的行
        rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr")

        # 遍历每一行提取数据
        data = []
        for row in rows:
            columns = row.find_elements(By.TAG_NAME, "td")
            row_data = [col.text.strip() for col in columns]
            data.append(row_data)
        # 将数据添加到 reduce_data 列表中
        reduce_data.extend(data)
       # 翻页部分
        try:
            pagination_items = driver.find_elements(By.CSS_SELECTOR, "ul.pagination li")
            next_page_btn = pagination_items[-1].find_element(By.TAG_NAME, "a")
            next_page_btn.click()
            time.sleep(random.randint(1, 3))
        except Exception:
            return 
    return reduce_data

In [10]:
form_data=get_data()

正在抓取第1页
正在抓取第2页
正在抓取第3页
正在抓取第4页
正在抓取第5页


# 数据清洗

In [12]:
import pandas as pd
# 转化为 DataFrame
columns = ['name', 'size', 'grade', 'location', 'category', 'price', 'date']
df = pd.DataFrame(form_data, columns=columns)
df

,name,size,grade,location,category,price,date
0,双面热压拼板机,6200-1300-180mm,优等,山东,板材原木,110000元/台,2024-05-15
1,木工热压机,2500-1300-42mm,优等,山东,板材原木,110000元/台,2024-05-15
2,可调速单板剪切机,2700-60mm,优等,山东,板材原木,23000元/台,2024-05-15
3,双面A字形拼方机,6200-1300-150mm,优等,山东,板材原木,110000元/台,2024-05-15
4,各种规格木业裁皮机,2700-60mm,优等,山东,板材原木,23000元/台,2024-05-15
...,...,...,...,...,...,...,...
95,南美菠萝格板材,规格可订,优等,进口,板材原木,4000元/m3,2023-02-08
96,柳桉全红板材,规格可订,优等,进口,板材原木,3000元/m3,2023-02-08
97,菠萝格全红板材,规格可订,优等,进口,板材原木,7000元/m3,2023-02-08
98,巴蒂木地板,规格可订,优等,巴西,板材原木,4000元/m3,2023-02-08


In [13]:
# 移除grade为空的行
df = df[df['grade'] != '']

# 清洗日期转换为时间戳
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].astype(int) // 10**9

# 清洗grade 把 'a' 替换成 'A级'
df['grade'] = df['grade'].apply(lambda x: x.replace('a', 'A级') if x == 'a' else x)

# 匹配 '*等品' 的替换成	'*等'
df['grade'] = df['grade'].apply(lambda x: x.replace('等品', '等') if '等品' in x else x)

# 匹配仅有一个 '' 的替换成	'A级'
df['grade'] = df['grade'].apply(lambda x: x.replace('A', 'A级') if x == 'A' else x)

# 清洗仅有 '1' 的替换成 '一级'
df['grade'] = df['grade'].apply(lambda x: x.replace('1', '一级') if x == '1' else x)

# 从price列提取 '0.45元/棵' 的'棵'，并分组统计
df['type'] = df['price'].apply(lambda x: x.split('/')[1])


In [14]:
# 统计grade的种类然后列出来
df['grade'].value_counts()

grade
优等         35
A级         21
AAA        20
统材          6
FAS         3
B级          3
C级          3
特等          2
无节材         2
AA          1
通货级         1
带二接头通货级     1
一等          1
AB          1
Name: count, dtype: int64

In [15]:
# 统计location的种类然后列出来
df['location'].value_counts()

location
国产           20
山东           14
进口           14
东南亚          10
缅甸            6
非洲            6
欧洲            5
南美洲           5
东北牡丹江         4
湖南            3
北美洲           3
南美            2
巴西            2
贵州            2
老挝            2
南&#16047;     1
巴新            1
Name: count, dtype: int64

In [16]:
# 统计name的种类然后列出来
df['name'].value_counts()

name
楸木木皮       7
水曲柳木皮      5
金丝柚        4
节松木皮       3
南美菠萝格      2
          ..
鹅掌揪        1
美国小叶樱桃木    1
老挝大红酸枝木    1
山樟         1
巴蒂木板材      1
Name: count, Length: 79, dtype: int64

In [17]:
# 统计type的种类然后列出来
df['type'].value_counts()

type
m3    59
m2    19
台     14
吨      8
Name: count, dtype: int64

In [18]:
df

,name,size,grade,location,category,price,date,type
0,双面热压拼板机,6200-1300-180mm,优等,山东,板材原木,110000元/台,1715731200,台
1,木工热压机,2500-1300-42mm,优等,山东,板材原木,110000元/台,1715731200,台
2,可调速单板剪切机,2700-60mm,优等,山东,板材原木,23000元/台,1715731200,台
3,双面A字形拼方机,6200-1300-150mm,优等,山东,板材原木,110000元/台,1715731200,台
4,各种规格木业裁皮机,2700-60mm,优等,山东,板材原木,23000元/台,1715731200,台
...,...,...,...,...,...,...,...,...
95,南美菠萝格板材,规格可订,优等,进口,板材原木,4000元/m3,1675814400,m3
96,柳桉全红板材,规格可订,优等,进口,板材原木,3000元/m3,1675814400,m3
97,菠萝格全红板材,规格可订,优等,进口,板材原木,7000元/m3,1675814400,m3
98,巴蒂木地板,规格可订,优等,巴西,板材原木,4000元/m3,1675814400,m3


## 入库

In [118]:
import sqlite3

# 连接到 SQLite 数据库（如果数据库不存在会自动创建）
conn = sqlite3.connect('timber_data.db')
cursor = conn.cursor()

# 创建表格
cursor.execute('''
CREATE TABLE IF NOT EXISTS timber_data (
	name TEXT,
	size TEXT,
	grade TEXT,
	location TEXT,
	category TEXT,
	price TEXT,
	date TEXT,
	type TEXT
)
''')

# 将 DataFrame 数据插入到表格中
for row in df.itertuples(index=False):
	cursor.execute('''
	INSERT INTO timber_data (name, size, grade, location, category, price, date,	type)
	VALUES (?, ?, ?, ?, ?, ?, ?,	?)
	''', row)

# 提交事务并关闭连接
conn.commit()
conn.close()

# ===============================================================

# 自动化全流程爬取

In [16]:
# 将下载的浏览器驱动文件解压并放在一个好记的文件夹中
import os
driver_path=''
match(os.name):
    case "nt":
        # Windows
        driver_path = r"./driver/chromedriver-win64/chromedriver.exe"  
        print("Windows")
    case "posix":
        # Mac
        driver_path = r"./driver/chromedriver-mac-arm64/chromedriver"  
        print("Mac")

os.environ["PATH"] += ";" + driver_path

import sqlite3
# 连接到 SQLite 数据库（如果数据库不存在会自动创建）
conn = sqlite3.connect('timber_data.db')
cursor = conn.cursor()
# 创建表格
cursor.execute('''
CREATE TABLE IF NOT EXISTS timber_data (
	name TEXT,
	size TEXT,
	grade TEXT,
	location TEXT,
	category TEXT,
	price TEXT,
	date TEXT,
	type TEXT
)
''')

Mac


In [17]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
driver = webdriver.Chrome(service=Service(executable_path=driver_path))
driver.get("https://www.chinatimber.org/baojia/baojia.html?prov=&bigCate=&q=")
time.sleep(3)

In [ ]:
import pandas as pd
import random
from selenium.webdriver.common.by import By
# 定位到倒数第二个 <li> 元素 即为最大页数
pagination_items = driver.find_elements(By.CSS_SELECTOR, "ul.pagination li")
max_page_item = pagination_items[-2] 
# 提取文本
max_page = max_page_item.text.strip()
print(f"数据最大页数为： {max_page}")
max_page=int(max_page)
reduce_data = []
def get_data():
    for i in range(1, int(max_page) + 1):
        print(f"正在抓取第{i}页")
        # 定位表格中的行
        rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr")

        # 遍历每一行提取数据
        data = []
        for row in rows:
            columns = row.find_elements(By.TAG_NAME, "td")
            row_data = [col.text.strip() for col in columns]
            data.append(row_data)

        # 转化为 DataFrame
        columns = ['name', 'size', 'grade', 'location', 'category', 'price', 'date']
        df = pd.DataFrame(data, columns=columns)
        df = df[df['grade'] != '']  # 移除 grade 为空的行
        
        # 数据清洗逻辑...
        # 清洗日期转换为时间戳
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df = df.dropna(subset=['date'])
        df['date'] = df['date'].astype(int) // 10**9

        # 清洗 grade
        df['grade'] = df['grade'].apply(lambda x: x.replace('a', 'A级') if x == 'a' else x)
        df['grade'] = df['grade'].apply(lambda x: x.replace('等品', '等') if '等品' in x else x)
        df['grade'] = df['grade'].apply(lambda x: x.replace('A', 'A级') if x == 'A' else x)
        df['grade'] = df['grade'].apply(lambda x: x.replace('1', '一级') if x == '1' else x)
        df['type'] = df['price'].apply(lambda x: x.split('/')[1] if '/' in x else '无单位')


        # 插入数据到数据库
        insert_data(df)

        # 翻页部分
        try:
            pagination_items = driver.find_elements(By.CSS_SELECTOR, "ul.pagination li")
            next_page_btn = pagination_items[-1].find_element(By.TAG_NAME, "a")
            next_page_btn.click()
            time.sleep(random.randint(1, 3))
        except Exception:
            return 
        
def insert_data(df):
    for _, row in df.iterrows():
        cursor.execute('''
        INSERT INTO timber_data (name, size, grade, location, category, price, date, type)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', tuple(row))  # 将每一行数据转为元组传入
    conn.commit()  # 提交事务



form_data=get_data()
form_data

In [19]:
conn.close()